# ONLINE RETAIL MARKET BASKEKT ANALYSIS

In [ ]:
!pip install openpyxl
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5973 sha256=4480df247baa5cb032652f022d033d6d3d86b673b1e0c69e5ba8f2155375f391
  Stored in directory: /root/.cache/pip/wheels/1b/02/6c/a45230be8603bd95c0a51cd2b289aefdd860c1a100eab73661
Successfully built apyori


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
#import utils
%matplotlib inline

# Import Warnings:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [ ]:
#Load dataset
OR = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx', sheet_name='Online Retail')

In [ ]:

# Set notebook Configuration:

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
#Make copy of the dataset
OR_copy = OR

In [ ]:
OR.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000,541909.000,406829.000
mean,9.552,4.611,15287.691
std,218.081,96.760,1713.600
min,-80995.000,-11062.060,12346.000
25%,1.000,1.250,13953.000
50%,3.000,2.080,15152.000
75%,10.000,4.130,16791.000
max,80995.000,38970.000,18287.000


In [ ]:
OR_copy.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000,541909.000,406829.000
mean,9.552,4.611,15287.691
std,218.081,96.760,1713.600
min,-80995.000,-11062.060,12346.000
25%,1.000,1.250,13953.000
50%,3.000,2.080,15152.000
75%,10.000,4.130,16791.000
max,80995.000,38970.000,18287.000


In [ ]:
OR.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom


In [ ]:
OR.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.850,12680.000,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.100,12680.000,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.150,12680.000,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.150,12680.000,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.950,12680.000,France


In [ ]:
#OR = OR_copy

In [ ]:
OR.info() #Summarise the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
OR.isna().sum() #search for missing values

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
OR.shape #Confirm

(541909, 8)

In [ ]:
OR.dropna(inplace=True)

In [ ]:
OR.shape

(406829, 8)

In [ ]:
OR.describe()

,Quantity,UnitPrice,CustomerID
count,406829.000,406829.000,406829.000
mean,12.061,3.460,15287.691
std,248.693,69.315,1713.600
min,-80995.000,0.000,12346.000
25%,2.000,1.250,13953.000
50%,5.000,1.950,15152.000
75%,12.000,3.750,16791.000
max,80995.000,38970.000,18287.000


In [ ]:
# Filter Transactions with cancelled status (Invoice NO contains value "C"):
OR_InvoiceNo = pd.DataFrame({"InvoiceNo":[row for row in OR["InvoiceNo"].values if "C"  not in str(row)]})
OR_InvoiceNo.head()

,InvoiceNo
0,536365
1,536365
2,536365
3,536365
4,536365


In [ ]:
#Drop duplicates:
OR_InvoiceNo = OR_InvoiceNo.drop_duplicates("InvoiceNo") 

In [ ]:
#The transactions except cancelled transactions:
OR = OR.merge(OR_InvoiceNo, on = "InvoiceNo")

In [ ]:
OR.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000,397924.000,397924.000
mean,13.022,3.116,15294.315
std,180.420,22.097,1713.170
min,1.000,0.000,12346.000
25%,2.000,1.250,13969.000
50%,6.000,1.950,15159.000
75%,12.000,3.750,16795.000
max,80995.000,8142.750,18287.000


In [ ]:
#The minimum values present in the 'Quantity' COlumn shows that we have outliers in the data,so we proceed to filter the numeric values to hold a minimum of 0. 
OR = OR[OR["Quantity"] > 0]
OR = OR[OR["UnitPrice"] > 0]

In [ ]:
OR.describe()

,Quantity,UnitPrice,CustomerID
count,397884.000,397884.000,397884.000
mean,12.988,3.116,15294.423
std,179.332,22.098,1713.142
min,1.000,0.001,12346.000
25%,2.000,1.250,13969.000
50%,6.000,1.950,15159.000
75%,12.000,3.750,16795.000
max,80995.000,8142.750,18287.000


In [ ]:
OR.Description.nunique() #Confirm count of unique values in 'Description' column

3877

In [ ]:
OR.StockCode.nunique() #Confirm count of unique values in 'StockCode' column

3665

In [ ]:
#The Unique values of these 2 variables (Description & StockCode) should be equal, because they are unique identifiers for each product.
#This means that each stock code/description holds a unique representation of each product.
#Hence we proceed to balance the both columns by dropping their respective duplicates:

#Review Stockcode values relative to Description:
OR_product = OR[["Description","StockCode"]].drop_duplicates() #drop duplicates
OR_product = OR_product.groupby(["Description"]).agg({"StockCode":"count"}).reset_index()
OR_product.sort_values("StockCode", ascending=False).head()

#The table below shows we still have duplicated stock codes

,Description,StockCode
2014,"METAL SIGN,CUPCAKE SINGLE HOOK",3
2471,PINK FLOWERS RABBIT EASTER,2
1077,EAU DE NILE JEWELLED PHOTOFRAME,2
2460,PINK FAIRY CAKE CUSHION COVER,2
3678,WHITE BAMBOO RIBS LAMPSHADE,2


In [ ]:
#Delete products with more than one stock code:
OR_product.rename(columns={'StockCode':'StockCode_Count'},inplace=True)
OR_product = OR_product[OR_product["StockCode_Count"]>1]
OR = OR[~OR["Description"].isin(OR_product["Description"])]

In [ ]:
#Review unique values:
print(OR.StockCode.nunique())
print(OR.Description.nunique())

3630
3858


In [ ]:
#Review Description values relative to StockCode:
OR_product = OR[["Description","StockCode"]].drop_duplicates()
OR_product = OR_product.groupby(["StockCode"]).agg({"Description":"count"}).reset_index()
OR_product.rename(columns={'Description':'Description_Count'},inplace=True)

In [ ]:
#Delete Stock codes representing multiple product description:
OR_product = OR_product.sort_values("Description_Count", ascending=False)
OR_product.head() #Review Count of Product Descriptions

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


In [ ]:
OR_product = OR_product[OR_product["Description_Count"] > 1] 
OR_product.head()

,StockCode,Description_Count
1990,23236,4
1950,23196,4
2104,23366,3
1886,23131,3
1994,23240,3


In [ ]:
OR = OR[~OR["StockCode"].isin(OR_product["StockCode"])]

In [ ]:
# Review unique values of Stock code and Product Description to confirm duplicates have been dropped.Now each stock code represents a single product:

print(OR.StockCode.nunique())
print(OR.Description.nunique())

3420
3420


In [ ]:
#We can see that each stock code represents a single product. 
#We will proceed to review the contents of the stock code called 'POST' and 'BANK CHARGES' as these are costs not products:
OR = OR[~OR["StockCode"].str.contains("POST", na=False)]
OR = OR[~OR["StockCode"].str.contains("BANK CHARGES", na=False)]

In [ ]:
#Save a copy of the preprocessed data:
OR_preprocessed = OR

In [ ]:
#Filter the data to sample sales in United Kingdom, for the purpose of demonstration:
OR_France = OR[OR["Country"] == "France"]
OR_France.shape

(7316, 8)

###  Further Preprocess the data to fit apriori model structure.

In [ ]:
#Define function to render a collection of transactions represented by their InvoiceNo, StockCode or Description and Quantity ordered in binary format to indicate purchase:

def invoice_function(dataframe, id=False):
    if id:
        return dataframe.groupby(['InvoiceNo', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

In [ ]:
Inv = invoice_function(OR_France, id=True)
Inv.head()

StockCode,10002,10120,10125,11001,15036,15039,16012,16048,16218,16219,16225,16236,16237,16238,17174,20615,20617,20658,20665,20668,20674,20675,20676,20677,20679,20681,20682,20684,20685,20686,20702,20704,20711,20712,20713,20717,20718,20719,20723,20724,20726,20727,20728,20749,20750,20751,20767,20777,20819,20832,20866,20961,20963,20966,20967,20971,20972,20973,20974,20975,20977,20978,20979,20981,20983,20984,20992,20996,21012,21026,21027,21030,21034,21035,21039,21042,21054,21055,21056,21058,21059,21061,21062,21063,21064,21065,21068,21069,21070,21078,21080,21084,21086,21087,21088,21090,21094,21095,21096,21098,21108,21110,21111,21114,21116,21121,21122,21123,21124,21125,21126,21135,21136,21137,21154,21155,21156,21161,21162,21163,21164,21165,21166,21174,21181,21188,21194,21195,21196,21197,21198,21199,21200,21201,21202,21204,21205,21206,21207,21208,21209,21210,21212,21213,21216,21217,21218,21219,21221,21224,21231,21238,21239,21240,21242,21244,21245,21246,21248,21249,21251,21257,21258,21259,21262,21284,21286,21294,21306,21326,21328,21329,21340,21351,21354,21355,21356,21358,21361,21365,21369,21371,21372,21374,21375,21377,21380,21381,21383,21385,21394,21395,21398,21399,21402,21403,21407,21408,21411,21424,21425,21426,21427,21428,21429,21430,21439,21442,21452,21463,21466,21467,21469,21470,21471,21472,21479,21481,21484,21485,21494,21495,21497,21498,21499,21500,21506,21508,21509,21519,21523,21524,21527,21528,21531,21533,21535,21537,21539,21544,21555,21556,21557,21558,21559,21561,21563,21564,21576,21577,21578,21579,21581,21584,21591,21621,21622,21623,21624,21643,21648,21650,21658,21668,21669,21671,21672,21673,21675,21677,21678,21679,21680,21693,21696,21698,21700,21703,21704,21706,21707,21708,21709,21710,21711,21715,21716,21719,21721,21723,21724,21725,21731,21733,21739,21746,21747,21749,21754,21755,21756,21770,21774,21786,21787,21790,21791,21812,21813,21819,21821,21828,21829,21832,21833,21843,21844,21845,21864,21865,21866,21871,21872,21875,21877,21880,21881,21882,21883,21884,21888,21889,21890,21891,21892,21894,21900,21901,21905,21906,21907,21908,21912,21913,21914,21915,21916,21917,21918,21922,21926,21929,21930,21931,21932,21933,21934,21935,21936,21937,21943,21946,21947,21948,21949,21955,21967,21972,21974,21975,21976,21977,21980,21981,21982,21983,21985,21986,21987,21988,21989,21990,22023,22024,22025,22026,22027,22028,22029,22032,22035,22037,22041,22043,22044,22045,22046,22051,22052,22059,22063,22064,22065,22068,22070,22071,22072,22073,22076,22077,22078,22079,22080,22081,22082,22083,22084,22085,22086,22087,22088,22089,22090,22091,22093,22094,22099,22109,22110,22111,22112,22113,22114,22115,22116,22118,22120,22121,22124,22130,22131,22132,22133,22136,22138,22139,22141,22142,22144,22149,22150,22151,22153,22170,22174,22175,22176,22178,22180,22181,22189,22191,22192,22193,22195,22198,22200,22203,22204,22207,22208,22209,22210,22211,22215,22219,22221,22222,22223,22224,22228,22231,22232,22236,22242,22243,22244,22254,22255,22262,22264,22269,22271,22272,22273,22274,22282,22294,22295,22299,22300,22301,22302,22303,22305,22306,22309,22311,22312,22313,22314,22315,22318,22319,22320,22321,22322,22324,22325,22326,22327,22328,22329,22331,22332,22333,22334,22339,22340,22341,22343,22344,22346,22348,22349,22350,22352,22354,22355,22356,22357,22358,22361,22362,22365,22366,22367,22371,22372,22374,22375,22376,22377,22378,22379,22380,22381,22382,22384,22385,22386,22389,22390,22393,22394,22396,22398,22399,22400,22402,22403,22406,22408,22411,22412,22413,22414,22417,22418,22419,22422,22423,22425,22427,22431,22432,22433,22434,22435,22436,22437,22438,22439,22440,22443,22444,22445,22449,22451,22452,22453,22454,22456,22457,22459,22467,22468,22469,22471,22472,22474,22475,22480,22482,22483,22485,22489,22491,22492,22493,22495,22498,22499,22500,22501,22504,22505,22507,22508,22514,22515,22516,22517,22518,22519,22520,22521,22522,22523,22524,22525,22526,22529,22530,22531,22532,22534,22535,22537,22539,22540,22541,22544,22546,22548,22549,22550,22551,22553,22554,22555,22556,22557,22558,22559,

In [ ]:
# Let's define a function to review and select the product name corresponding to the stock code:

def id_review(dataframe, stockcode):
    product_name = dataframe[dataframe["StockCode"] == stockcode]["Description"].unique()[0]
    return stockcode, product_name

In [ ]:
id_review(OR_France, 10002)

(10002, 'INFLATABLE POLITICAL GLOBE ')

In [ ]:
# Import Warnings:
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

# For Association Rules Learning & Apriori 
!pip install mlxtend
from mlxtend.frequent_patterns import apriori, association_rules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
frequent_itemsets = apriori(Inv, min_support=0.02, use_colnames=True)

In [ ]:
frequent_itemsets.head()

,support,itemsets
0,0.021,(10002)
1,0.040,(20676)
2,0.021,(20677)
3,0.034,(20679)
4,0.071,(20682)


In [ ]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.02)

rules.sort_values("support", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
277,(21094),(21086),0.132,0.143,0.127,0.960,6.720,0.108,21.429
276,(21086),(21094),0.143,0.132,0.127,0.889,6.720,0.108,7.810
939,(22551),(22554),0.140,0.177,0.108,0.774,4.364,0.084,3.634
938,(22554),(22551),0.177,0.140,0.108,0.612,4.364,0.084,2.216
995,(22556),(22554),0.175,0.177,0.106,0.606,3.419,0.075,2.089
230,(21080),(21086),0.138,0.143,0.106,0.769,5.385,0.086,3.714
231,(21086),(21080),0.143,0.138,0.106,0.741,5.385,0.086,3.327
232,(21080),(21094),0.138,0.132,0.106,0.769,5.815,0.088,3.760
233,(21094),(21080),0.132,0.138,0.106,0.800,5.815,0.088,4.312
994,(22554),(22556),0.177,0.175,0.106,0.597,3.419,0.075,2.048


In [ ]:
rules.sort_values("lift", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4014,"(22728, 21086)","(21094, 22726)",0.026,0.021,0.021,0.800,37.800,0.021,4.894
4019,"(21094, 22726)","(22728, 21086)",0.021,0.026,0.021,1.000,37.800,0.021,inf
4016,"(22728, 21094)","(22726, 21086)",0.024,0.024,0.021,0.889,37.333,0.021,8.786
4017,"(22726, 21086)","(22728, 21094)",0.024,0.024,0.021,0.889,37.333,0.021,8.786
3493,(23054),"(23049, 23052)",0.026,0.024,0.021,0.800,33.600,0.021,4.881
3488,"(23049, 23052)",(23054),0.024,0.026,0.021,0.889,33.600,0.021,8.762
1219,(23054),(23049),0.026,0.032,0.024,0.900,28.350,0.023,9.683
1218,(23049),(23054),0.032,0.026,0.024,0.750,28.350,0.023,3.894
1223,(23054),(23052),0.026,0.032,0.024,0.900,28.350,0.023,9.683
1222,(23052),(23054),0.032,0.026,0.024,0.750,28.350,0.023,3.894


In [ ]:
rules.sort_values("confidence", ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3255,"(22554, 23199)",(23206),0.024,0.130,0.024,1.000,7.714,0.021,inf
4165,"(22352, 22551, 22303)",(22556),0.021,0.175,0.021,1.000,5.727,0.017,inf
5248,"(22728, 22554, 22556, 22726)",(22551),0.021,0.140,0.021,1.000,7.132,0.018,inf
4472,"(22659, 22556, 22382)",(22551),0.024,0.140,0.024,1.000,7.132,0.020,inf
5337,"(22728, 22729, 22554, 22727)",(22551),0.021,0.140,0.021,1.000,7.132,0.018,inf
4626,"(22728, 22554, 22726)",(22551),0.029,0.140,0.029,1.000,7.132,0.025,inf
4612,"(22554, 22726, 22727)",(22551),0.026,0.140,0.026,1.000,7.132,0.023,inf
5307,"(22729, 22554, 22726, 22727)",(22551),0.021,0.140,0.021,1.000,7.132,0.018,inf
4598,"(22728, 22554, 22556)",(22551),0.026,0.140,0.026,1.000,7.132,0.023,inf
4556,"(22554, 22556, 22661)",(22551),0.026,0.140,0.026,1.000,7.132,0.023,inf


In [ ]:
sorted_rules = rules.sort_values("lift", ascending=False)

4.To ensure the usability of the determined association rules, the model was tested for product Suggestion to customers. At the product offer stage, we might devise many techniques.
For example, when X is purchased, we may sort based on the likelihood of purchasing Y (confidence) and make a product offer, or we can make an offer based on how many times the chance of sales grows over the lift. We may also produce a product suggestion using a hybrid filtering method that combines support, lift, and confidence.


We proceed to select a particular product to produce recommendations for based on the designed model:

In [ ]:
product_id = 22554

id_review(OR, product_id)

(22554, 'PLASTERS IN TIN WOODLAND ANIMALS')

In [ ]:
product_id = 22726
recommendation_list = []


for idx, product in enumerate(sorted_rules["antecedents"]):
    # Since it is antecendent tuple, let's convert it to a list and search within the list:
    for j in list(product):
        if j == product_id:
            # Whatever the index of this integer value we have captured (idx), we will search for it in the consequent, let's suggest the first product [0] for the rows we find 
            recommendation_list.append(list(sorted_rules.iloc[idx]["consequents"])[0])
            recommendation_list = list( dict.fromkeys(recommendation_list) )

In [ ]:
list_top5 = recommendation_list[0:5]
list_top5

[22728, 22729, 22551, 22554, 22555]

In [ ]:
# Let's show the product names of top 5 recommended products:

for elem in list_top5:
    print(id_review(OR_France,elem))

(22728, 'ALARM CLOCK BAKELIKE PINK')
(22729, 'ALARM CLOCK BAKELIKE ORANGE')
(22551, 'PLASTERS IN TIN SPACEBOY')
(22554, 'PLASTERS IN TIN WOODLAND ANIMALS')
(22555, 'PLASTERS IN TIN STRONGMAN')


Having tested the model and confirmed it is working. This can be employed into the business's purchasing processes such that when a customer selects a particular product, a list of the top related products are offered or cross-sold from the results of the model.

In [ ]:
!pip install plotly==5.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.2 MB 7.3 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0


Plot the association rules:

In [ ]:
rules_df = pd.DataFrame(rules, columns=['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'leverage', 'conviction'])
import plotly.express as px
fig = px.scatter(rules_df, x="support", y="confidence", color="lift",
                hover_data=['antecedent support','consequent support'], color_continuous_scale="agsunset")
fig.show()